In [1]:
clear all

In [2]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy import io
import scipy.io as sio
%matplotlib inline 
import pylab
import csv
from Tkinter import Tk
from tkFileDialog import askopenfilename
from tkFileDialog import askdirectory
import nibabel as nb
from scipy import io
from nifti import NiftiImage
import nibabel as nb
from scipy.interpolate import interp1d
from scipy import ndimage

# Open Masks

In [3]:
# from http://stackoverflow.com/questions/3579568/choosing-a-file-in-python-with-simple-dialog
from Tkinter import Tk
from tkFileDialog import askopenfilename

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filenameM = askopenfilename() # show an "Open" dialog box and return the path to the selected file
print(filenameM)
img1 = nb.load(filenameM)
Masks = img1.get_data()
Sm=Masks.shape
Masks=np.array(Masks)

/home/sophie/Desktop/Registration/JFRCMasks8bit.nii


In [4]:
filenameM='/home/sophie/Desktop/Registration/RegionList'
with open(filenameM) as f:
    content = f.readlines()
Names=[Line.split('\t') for Line in content]
RegionName=[Names[i][0] for i in range(75)]
Num=[int(Names[i][2]) for i in range(75)]

In [5]:
Masks.shape

(1024, 512, 218, 86)

In [6]:
S=Masks.shape

In [7]:
len(Names)

75

# Reorder by larger sub-regions (presumed ~stimulus to motor)

In [8]:
LargerRegionsDic={'':'','AME_R':'OL','LO_R':'OL','NO':'CX','BU_R':'CX','PB':'CX','LH_R':'LH','LAL_R':'LX','SAD':'PENP'
               ,'CAN_R':'PENP','AMMC_R':'PENP','ICL_R':'INP','VES_R':'VMNP','IB_R':'INP','ATL_R':'INP','CRE_R':'INP'
               ,'MB_PED_R':'MB','MB_VL_R':'MB','MB_ML_R':'MB','FLA_R':'PENP','LOP_R':'OL','EB':'CX','AL_R':'AL',
                'ME_R':'OL','FB':'CX','SLP_R':'SNP','SIP_R':'SNP','SMP_R':'SNP','AVLP_R':'VLNP','PVLP_R':'VLNP',
                'IVLP_R':'VLNP','PLP_R':'VLNP','AOTU_R':'VLNP','GOR_R':'VMNP','MB_CA_R':'MB','SPS_R':'VMNP',
                'IPS_R':'VMNP','SCL_R':'INP','EPA_R':'VMNP','GNG':'GNG','PRW':'PENP','GA_R':'LX','AME_L':'OL'
                ,'LO_L':'OL','BU_L':'CX','LH_L':'LH','LAL_L':'LX','CAN_L':'PENP','AMMC_L':'PENP','ICL_L':'INP',
                'VES_L':'VMNP','IB_L':'INP','ATL_L':'INP','CRE_L':'INP','MB_PED_L':'MB','MB_VL_L':'MB',
                'MB_ML_L':'MB','FLA_L':'PENP','LOP_L':'OL','AL_L':'AL','ME_L':'OL','SLP_L':'SNP','SIP_L':'SNP',
                'SMP_L':'SNP','AVLP_L':'VLNP','PVLP_L':'VLNP','IVLP_L':'VLNP','PLP_L':'VLNP','AOTU_L':'VLNP',
                'GOR_L':'VMNP','MB_CA_L':'MB','SPS_L':'VMNP','IPS_L':'VMNP','SCL_L':'INP','EPA_L':'VMNP','GA_L':'LX'}

In [9]:
SmallRegionsSorted=['ME_L','ME_R','LO_R','LO_L','LOP_R','LOP_L','AME_R','AME_L',
                  'PLP_R','PLP_L','PVLP_R','PVLP_L','AVLP_R','AVLP_L','AOTU_R','AOTU_L','IVLP_R','IVLP_L',
                  'AL_R','AL_L',
                  'MB_CA_R','MB_CA_L','MB_PED_R','MB_PED_L','MB_VL_R','MB_VL_L','MB_ML_R','MB_ML_L',
                  'SMP_R','SMP_L','SIP_R','SLP_L','SLP_R','SIP_L',
                  'LH_R','LH_L',                  
                  'CRE_R','CRE_L','ICL_R','ICL_L','SCL_R','SCL_L','IB_R','IB_L',
                  'EB','ATL_R','ATL_L','PB','NO','FB',
                  'BU_R','BU_L','LAL_R','LAL_L','GA_R','GA_L',
                  'GOR_R','GOR_L','EPA_R','EPA_L','VES_R','VES_L','SPS_R','SPS_L','IPS_R','IPS_L',
                  'AMMC_R','AMMC_L','SAD','FLA_R','FLA_L','PRW','CAN_R','CAN_L',
                  'GNG','']

In [10]:
len(SmallRegionsSorted)

76

In [11]:
Tozip=range(len(SmallRegionsSorted))
SmallRegionsDic=dict(zip(SmallRegionsSorted,Tozip))

In [12]:
LargerRegionInd={ 'OL':1,'VLNP':2,'VMNP':3,'AL':4,'MB':5,'LH':6,'SNP':7,'CX':8,'LX':9,'INP':10,'PENP':11,'GNG':12,'':13}

In [13]:
LargerRegionMask=[LargerRegionsDic[Names[i][0]] for i in range(75)]
LargerRegionMaskId=np.zeros(87)
for i in range(75):
    LargerRegionMaskId[Num[i]]=np.int(LargerRegionInd[LargerRegionMask[i]])

In [14]:
LargerRegionMaskId

array([  0.,   0.,   1.,   1.,   8.,   8.,   8.,   6.,   9.,  11.,  11.,
        11.,  10.,   3.,  10.,  10.,  10.,   5.,   5.,   5.,  11.,   0.,
         1.,   8.,   4.,   1.,   8.,   7.,   7.,   7.,   2.,   2.,   2.,
         2.,   2.,   3.,   5.,   3.,   3.,  10.,   3.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,  12.,  11.,   9.,   1.,   1.,   8.,
         6.,   9.,  11.,  11.,  10.,   3.,  10.,  10.,  10.,   5.,   5.,
         5.,  11.,   0.,   1.,   4.,   1.,   7.,   7.,   7.,   2.,   2.,
         2.,   2.,   2.,   3.,   5.,   3.,   3.,  10.,   3.,   9.])

In [15]:
np.max(LargerRegionMaskId)

12.0

In [16]:
NewMasks=np.zeros((S[0],S[1],S[2],12))

In [18]:
for i in range(12):
    for j in range(75):
        k=0
        if LargerRegionMaskId[j]==i:            
            NewMasks[:,:,:,i]=NewMasks[:,:,:,i]+Masks[:,:,:,j]
            k=k+1
        NewMasks[:,:,:,i]=NewMasks[:,:,:,i]/k
    print(i)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in divide
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in divide


0
1
2
3
4
5
6
7
8
9
10
11


In [ ]:
#D4=np.transpose(data[:,:,:,range(ONint,(OFFint+1))],(2,1,0,3))
nim=nib.Nifti1Image(NewMasks,np.eye(4))
nib.save(nim,'/home/sophie/Desktop/Registration/LargeRegionsJFRC.nii')